<a href="https://colab.research.google.com/github/lolpa1n/FaceEvaluation/blob/master/FaceEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip uninstall -y tensorflow
!pip3 install tensorflow-gpu==1.14 

In [0]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/'My Drive'/crop_net_new.zip /content/sample_data/
!unzip -q "/content/sample_data/crop_net_new.zip"

In [0]:

import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.models import load_model

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.nasnet import NASNetMobile

In [0]:
def insert_in_background(image, size):
    im_h = image.shape[0]
    im_w = image.shape[1]
    
    bg_h, bg_w = size
    background =  np.zeros((bg_h, bg_w, 3), dtype=np.int)
    
    for i in range(3):
        background[:im_h, :im_w, i] = image[:,:, i]
        
    return background
  
def resize_and_padding(image, size):
    
    im_h = image.shape[0]
    im_w = image.shape[1]
    
    bg_h, bg_w = size
    background =  np.zeros((bg_h, bg_w, 3), dtype=np.int)
    
    if im_h > im_w:
        koef = im_h/bg_h
        
        new_w = int(im_w / koef)
        new_h = int(im_h / koef)
        
        image = cv2.resize(image, (new_w, new_h))
    else:
        koef = im_w/bg_w
        
        new_h = int(im_h / koef)
        new_w = int(im_w / koef)
        
        image = cv2.resize(image, (new_w, new_h))
        
    for i in range(3):
        background[:new_h, :new_w, i] = image[:,:, i]
    
    return background
  
def resize_image(image, size):
  return resize_and_padding(image, size)

      

import pathlib
new_dir = '/content/crop_net/'
pathlib.Path(new_dir).mkdir(parents=True, exist_ok=True) 

def convert_image(path, file):
  img = cv2.imread(path + file)
  img = resize_image(img, (224, 224))
  cv2.imwrite(new_dir + file, img)

In [0]:
IMG_H = 224
IMG_W = 224
IMG_C = 3

def read_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_W, IMG_H))
    if IMG_C == 1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.reshape(img.shape[0], img.shape[1], 1)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(float)
    img /= 255
    return img
  
directory = '/content/crop_net_new/'
files = os.listdir(directory)

data = pd.DataFrame(columns=['path', 'proba'])

for i, file in enumerate(files):
    info = file.split('_')
    views = int(info[2])
    likes = int(info[3])
    friends = int(info[4].split('.')[0])
    
    img = cv2.imread(directory + file)
    if img.shape[0] < IMG_H/3 and img.shape[1] < IMG_W/3:
      continue
    
    proba = 0.95 if likes / views > 0.95 else likes / views
    
    path = new_dir + file
    try:
      convert_image(directory, file)
      data = data.append({'path': path, 'proba': proba}, ignore_index = True)
    except:
      print(file)
data = data.drop(data[data['proba'] > 0.5].index)

In [0]:
data_train, data_test = train_test_split(data, test_size=0.1, random_state=17)

In [0]:
data_train.shape, data_test.shape

In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

batch_size = 100

train_datagen = ImageDataGenerator(rescale = 1./255) 

train_generator = train_datagen.flow_from_dataframe(dataframe=data_train,
                                              x_col="path", y_col="proba", has_ext=True, 
                                              class_mode="other", target_size=(IMG_H, IMG_W), 
                                              batch_size=batch_size)

test_datagen = ImageDataGenerator(rescale = 1./255) 

test_generator = test_datagen.flow_from_dataframe(dataframe=data_test,
                                              x_col="path", y_col="proba", has_ext=True, 
                                              class_mode="other", target_size=(IMG_H, IMG_W), 
                                              batch_size=batch_size)


In [0]:
from tensorflow.keras.optimizers import Adam, Adadelta, SGD

def create_model(layers, loss):
  pretrained_model = NASNetMobile(weights='imagenet', include_top=False)
  pretrained_model.trainable = True

  x = pretrained_model.output
  x = GlobalAveragePooling2D()(x)
  
  for layer in layers:
    x = Dense(layer)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

  predictions = Dense(1, activation='sigmoid')(x)
  model = Model(inputs=pretrained_model.input, outputs=predictions)
  model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=loss, metrics=['acc'])
  return model

In [0]:
MIN_LAYERS_COUNT = 1
MAX_LAYERS_COUNT = 2

MIN_NEURONS_COUNT = 300
MAX_NEURONS_COUNT = 500
NEURON_STEP = 100


LAYERS = []
LOSSES = ['mse', 'mae']

for layer_count in range(MIN_LAYERS_COUNT, MAX_LAYERS_COUNT):
  for neuron_count in range(MIN_NEURONS_COUNT, MAX_NEURONS_COUNT, NEURON_STEP):
    layers = []
    for _ in range(layer_count):
      layers.append(neuron_count)
      
    LAYERS.append(layers)



In [0]:
steps_per_epoch = data_train.shape[0]//batch_size
validation_steps = data_test.shape[0]//batch_size

for layers in LAYERS:
  for loss in LOSSES:
    print(f'Loss: {loss}, Layers: {layers}')
    model = create_model(layers, loss)
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=test_generator,
        validation_steps=validation_steps,
        epochs=10,
        verbose=1)

In [0]:
steps_per_epoch = data_train.shape[0]//batch_size
validation_steps = data_test.shape[0]//batch_size

model = create_model([1024, 512], 'mse')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_generator,
    validation_steps=validation_steps,
    epochs=10,
    verbose=1)

In [0]:
model.save('NASNetMobile_1024_512_mse.h5')

In [0]:
data_test[200:300]

In [0]:
from matplotlib import pyplot as plt

test_predicted = []

for i, row in data_test[200:300].iterrows():
  test_img = read_image(row['path'])
  # plt.imshow(test_img)
  # plt.show()
  test_img = test_img[None, :,:,:]
  predict = model.predict(test_img)[0][0]
  test_predicted.append(predict)
  print(f'{predict:0.5}')

In [0]:
start = 0
end = 100
count = end - start
x = list(range(count))

plt.style.use('dark_background')

plt.figure(figsize=(20, 10))
plt.plot(x, test_predicted[start:end], color='yellow')
plt.plot(x, data_test['proba'][start:end], color='green')
plt.grid(True)
plt.show()